In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests
import plotly.express as px

# Шаг 1

In [2]:
url = 'https://drive.google.com/uc?id=1QosQQ4RRNR9rkL4t7sB707h2Uy0XfYJe&export=download'
df = pd.read_csv(url)
df.describe

<bound method NDFrame.describe of                                      uuid platform  \
0    1de9ea66-70d3-4a1f-8735-df5ef7697fb9      web   
1    f149f542-e935-4870-9734-6b4501eaf614      web   
2    f149f542-e935-4870-9734-6b4501eaf614      web   
3    08f0ebd4-950c-4dd9-8e97-b5bdf073eed1      web   
4    08f0ebd4-950c-4dd9-8e97-b5bdf073eed1      web   
..                                    ...      ...   
995  8bc047ac-dc03-4a61-8037-7371f729fa34      web   
996  8bc047ac-dc03-4a61-8037-7371f729fa34      web   
997  3f78ac76-6f81-43ec-85e8-f3cf74fc8fdc      web   
998  3f78ac76-6f81-43ec-85e8-f3cf74fc8fdc      web   
999  1408b7ab-6450-4337-b57d-fd8a64b7fe97      web   

                                            user_agent                 date  
0    Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...  2023-03-01T13:29:22  
1    Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...  2023-03-01T16:44:28  
2    Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...  2023-03-06T06:12:36  
3    

In [3]:
url = 'https://drive.google.com/uc?id=1AeQz0kaSgz0lxYSDtuNm36muhy5fRCzZ&export=download'
df = pd.read_csv(url)
df.describe

<bound method NDFrame.describe of                     date  user_id                    email platform  \
0    2023-03-01T00:25:39  8838849     joseph95@example.org      web   
1    2023-03-01T14:53:01  8741065  janetsuarez@example.net      web   
2    2023-03-01T14:27:36  1866654     robert67@example.org      web   
3    2023-03-01T02:42:34  1577584         elam@example.net      web   
4    2023-03-01T10:27:14  4765395  stephanie68@example.net      web   
..                   ...      ...                      ...      ...   
995  2023-03-05T03:19:40  6414793  jerryrivera@example.com  android   
996  2023-03-05T15:03:48    53167      laura65@example.net  android   
997  2023-03-05T09:51:08  4969623    jasmine42@example.net  android   
998  2023-03-05T16:25:59  7636873  imclaughlin@example.net  android   
999  2023-03-05T11:17:41  7551697  jacobgarcia@example.org  android   

    registration_type  
0              google  
1              yandex  
2              google  
3               a

## Шаг 2

In [14]:
begin_date = "2023-03-01"
end_date = "2023-09-01"
base_url = "https://data-charts-api.hexlet.app"

visits_url = f"{base_url}/visits?begin={begin_date}&end={end_date}"
registrations_url = f"{base_url}/registrations?begin={begin_date}&end={end_date}"

visits_response = requests.get(visits_url)
visits_data = visits_response.json()

registrations_response = requests.get(registrations_url)
registrations_data = registrations_response.json()

print(visits_data[0:5])
print("\n")
print(registrations_data[0:5])

[{'visit_id': '1de9ea66-70d3-4a1f-8735-df5ef7697fb9', 'platform': 'web', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.6', 'datetime': '2023-03-01T10:36:22'}, {'visit_id': 'f149f542-e935-4870-9734-6b4501eaf614', 'platform': 'web', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Geck', 'datetime': '2023-03-01T06:25:00'}, {'visit_id': '08f0ebd4-950c-4dd9-8e97-b5bdf073eed1', 'platform': 'web', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9', 'datetime': '2023-03-01T10:26:13'}, {'visit_id': '19322fed-157c-49c6-b16e-2d5cabeb9592', 'platform': 'web', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36', 'datetime': '2023-03-01T12:33:06'}, {'visit_id': '04762a22-3c9f-40c9-9ac9-6628c4381836', 'platform': '

# Шаг 3

In [18]:
visits_df = pd.DataFrame(visits_data)
registrations_df = pd.DataFrame(registrations_data)

visits_df.rename(columns={'datetime': 'date'}, inplace=True)
registrations_df.rename(columns={'datetime': 'date'}, inplace=True)

visits_df = visits_df[~visits_df['user_agent'].str.lower().str.contains('bot')]

visits_df['date'] = pd.to_datetime(visits_df['date'])
registrations_df['date'] = pd.to_datetime(registrations_df['date'])

visits_df = visits_df.sort_values('date').drop_duplicates(subset='visit_id', keep='last')

visits_grouped = (
    visits_df
    .groupby(['date', 'platform'])
    .agg(visits=('visit_id', 'count'))
    .reset_index()
)

registrations_grouped = (
    registrations_df
    .groupby(['date', 'platform'])
    .agg(registrations=('user_id', 'count'))
    .reset_index()
)

conversion_df = pd.merge(visits_grouped, registrations_grouped,
                         on=['date', 'platform'], how='outer')

conversion_df[['visits', 'registrations']] = conversion_df[['visits', 'registrations']].fillna(0)

conversion_df['conversion'] = (conversion_df['registrations'] / conversion_df['visits']) * 100
conversion_df['conversion'] = conversion_df['conversion'].round(6)

conversion_df.rename(columns={'date': 'date_group'}, inplace=True)

conversion_df = conversion_df.sort_values('date_group')

conversion_df['date_group'] = conversion_df['date_group'].astype('int64') // 10**6

conversion_df.to_json('conversion.json')

print(conversion_df.head())

      date_group platform  visits  registrations  conversion
0  1677628988000      web     1.0            0.0         0.0
1  1677629037000      web     1.0            0.0         0.0
2  1677629087000      web     0.0            1.0         inf
3  1677629490000      web     1.0            0.0         0.0
4  1677630180000  android     0.0            1.0         inf
